In [0]:
%tensorflow_version 2.x
import tensorflow as tf 
tf.test.gpu_device_name()

In [0]:

import os
import pprint
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from string import digits

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
os.listdir('/content/drive/My Drive/Project files')

['P1 stacksample.zip',
 'P2 genres.tar.gz',
 'P3 Quora Text Classification with embeddings.zip',
 'P4 state-farm-distracted-driver-detection(1).zip',
 'P5 Image Captioning.zip']

In [0]:
!mkdir Text

In [0]:
!unzip -q '/content/drive/My Drive/Project files/P3 Quora Text Classification with embeddings.zip' -d'/content/Text'

mapname:  conversion of  failed


In [0]:
Text=pd.read_csv('/content/Text/train.csv')

In [0]:
Text['target'].value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [0]:
Text.columns

Index(['qid', 'question_text', 'target'], dtype='object')

In [0]:
Text['question_text']

0          How did Quebec nationalists see their province...
1          Do you have an adopted dog, how would you enco...
2          Why does velocity affect time? Does velocity a...
3          How did Otto von Guericke used the Magdeburg h...
4          Can I convert montra helicon D to a mountain b...
                                 ...                        
1306117    What other technical skills do you need as a c...
1306118    Does MS in ECE have good job prospects in USA ...
1306119                            Is foam insulation toxic?
1306120    How can one start a research project based on ...
1306121    Who wins in a battle between a Wolverine and a...
Name: question_text, Length: 1306122, dtype: object

In [0]:
Text

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [0]:
import re

In [0]:
from nltk.stem import WordNetLemmatizer
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
Tok = Tokenizer()
lemma = WordNetLemmatizer()
my_stop=set(stopwords.words('english')+list(punctuation))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [0]:
list(punctuation)

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [0]:
def preprocessing(b):

  b=str(b)
  b=b.lower()
  b=re.sub(r"what's",'what is',b )
  b=re.sub(r"i've",'i have',b)
  b=re.sub(r"can't",'can not',b)
  b=re.sub(r"\'\n","  ",b)
  b=re.sub(r"\'s",' is',b)
  b=re.sub(r"\'ve"," have",b)
  b=re.sub(r"\'r",' are',b)
  b=re.sub(r"\'m",' am',b)
  b=re.sub(r"\'d",' would',b)
  b=re.sub(r"\'cause"," because",b)
  b=re.sub(r"\'\n","   ",b)
  b=re.sub('[^a-zA-Z]',' ', b)
  b=b.strip(" ")
  return b

In [0]:
def split_into_lemmas(x):
  
  words=x.lower()
  #for word in words:
    
  words= nltk.word_tokenize(words)
  #print(words)
  #regex = re.compile('[%s]' % re.escape(Punct))
  
  word_san_stop = []
  for word in words:
    
    if word not in my_stop:
      if len(word)>2 :
        word=lemma.lemmatize(word)
        word_san_stop.append(word)
  return ' '.join(word_san_stop)



In [0]:
e=Text['question_text'][0:2].apply(lambda x :preprocessing(x))
e

0    how did quebec nationalists see their province...
1    do you have an adopted dog  how would you enco...
Name: question_text, dtype: object

In [0]:
e.apply(lambda x :split_into_lemmas(x))

0           quebec nationalist see province nation
1    adopted dog would encourage people adopt shop
Name: question_text, dtype: object

In [0]:
Text['question_text']=Text['question_text'].apply(lambda x :preprocessing(x))

In [0]:
Text['question_text']=Text['question_text'].apply(lambda x :split_into_lemmas(x))

In [0]:
Text['question_text']

0                     quebec nationalist see province nation
1              adopted dog would encourage people adopt shop
2          velocity affect time velocity affect space geo...
3                otto von guericke used magdeburg hemisphere
4          convert montra helicon mountain bike changing ...
                                 ...                        
1306117      technical skill need computer science undergrad
1306118     ece good job prospect usa like india job present
1306119                                foam insulation toxic
1306120    one start research project based biochemistry ...
1306121                            win battle wolverine puma
Name: question_text, Length: 1306122, dtype: object

In [0]:
X_train,X_test,y_train,y_test=train_test_split(Text['question_text'],Text['target'],test_size=0.3,stratify=Text['target'])

In [0]:
sen_lens=[]
for sen in X_train:
  sen_lens.append(len(sen))
sen_lens

[36,
 49,
 88,
 25,
 44,
 57,
 36,
 19,
 22,
 24,
 17,
 31,
 71,
 30,
 23,
 49,
 49,
 47,
 34,
 76,
 29,
 51,
 22,
 28,
 86,
 20,
 19,
 68,
 41,
 35,
 8,
 57,
 41,
 31,
 19,
 28,
 41,
 35,
 31,
 42,
 28,
 20,
 71,
 26,
 32,
 25,
 33,
 20,
 160,
 37,
 35,
 34,
 31,
 36,
 21,
 30,
 30,
 47,
 47,
 20,
 62,
 61,
 25,
 70,
 54,
 92,
 45,
 60,
 27,
 42,
 58,
 18,
 50,
 38,
 27,
 53,
 65,
 128,
 44,
 44,
 35,
 72,
 61,
 63,
 64,
 55,
 30,
 28,
 24,
 28,
 22,
 34,
 48,
 39,
 39,
 26,
 100,
 42,
 20,
 24,
 31,
 37,
 112,
 23,
 30,
 43,
 34,
 38,
 66,
 16,
 4,
 65,
 62,
 79,
 46,
 86,
 44,
 38,
 45,
 61,
 43,
 23,
 28,
 39,
 105,
 45,
 28,
 15,
 46,
 27,
 13,
 43,
 32,
 68,
 14,
 91,
 59,
 50,
 46,
 22,
 55,
 22,
 60,
 43,
 36,
 60,
 74,
 29,
 34,
 40,
 31,
 25,
 35,
 38,
 31,
 41,
 31,
 84,
 79,
 23,
 44,
 40,
 158,
 39,
 34,
 133,
 40,
 42,
 23,
 100,
 47,
 39,
 96,
 26,
 24,
 26,
 29,
 24,
 43,
 121,
 28,
 50,
 52,
 17,
 22,
 35,
 103,
 103,
 26,
 58,
 32,
 27,
 58,
 54,
 22,
 28,
 31,
 25,
 

In [0]:

max_len=int(np.quantile(sen_lens,0.95))
#int(np.quantile(sen_lens,0.95))

In [0]:
tok=Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train)
Sequences_train=tok.texts_to_sequences(X_train)
Sequences_test=tok.texts_to_sequences(X_test)

In [0]:
tok.index_word

{1: 'get',
 2: 'best',
 3: 'would',
 4: 'people',
 5: 'like',
 6: 'good',
 7: 'one',
 8: 'make',
 9: 'india',
 10: 'year',
 11: 'way',
 12: 'time',
 13: 'think',
 14: 'many',
 15: 'life',
 16: 'much',
 17: 'someone',
 18: 'use',
 19: 'want',
 20: 'know',
 21: 'work',
 22: 'country',
 23: 'take',
 24: 'thing',
 25: 'job',
 26: 'woman',
 27: 'indian',
 28: 'ever',
 29: 'find',
 30: 'world',
 31: 'feel',
 32: 'person',
 33: 'become',
 34: 'book',
 35: 'without',
 36: 'could',
 37: 'better',
 38: 'quora',
 39: 'student',
 40: 'day',
 41: 'girl',
 42: 'difference',
 43: 'trump',
 44: 'mean',
 45: 'company',
 46: 'need',
 47: 'new',
 48: 'possible',
 49: 'school',
 50: 'college',
 51: 'start',
 52: 'used',
 53: 'friend',
 54: 'american',
 55: 'first',
 56: 'question',
 57: 'state',
 58: 'say',
 59: 'money',
 60: 'still',
 61: 'business',
 62: 'love',
 63: 'different',
 64: 'old',
 65: 'long',
 66: 'university',
 67: 'really',
 68: 'give',
 69: 'learn',
 70: 'help',
 71: 'change',
 72: 'right

In [0]:
Vocab_len=len(tok.index_word)
len(tok.index_word)

138188

In [0]:
S_matrix_train=sequence.pad_sequences(Sequences_train,maxlen=max_len)
S_matrix_test=sequence.pad_sequences(Sequences_test,maxlen=max_len)

In [0]:
S_matrix_test

array([[    0,     0,     0, ...,   464,    34, 16154],
       [    0,     0,     0, ...,   295,   497,    45],
       [    0,     0,     0, ...,    14,   136,    24],
       ...,
       [    0,     0,     0, ..., 11525,  1190,   953],
       [    0,     0,     0, ...,  1014,    14,    78],
       [    0,     0,     0, ...,   394,  2371,  1807]], dtype=int32)

In [0]:
y_train.value_counts()

0    857718
1     56567
Name: target, dtype: int64

In [0]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM,Dropout,Embedding,Dense,Input

In [0]:
model=Sequential()
model.add(Input(name='Input',shape=max_len))
model.add(Embedding(Vocab_len+1,512,mask_zero=True))
model.add(LSTM(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128,activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(128,activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))


In [0]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 93, 512)           70752768  
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)              

In [0]:
history=model.fit(S_matrix_train,y_train,validation_data=[S_matrix_test,y_test],batch_size=5000,epochs=5)

Train on 914285 samples, validate on 391837 samples
Epoch 1/5
914285/914285 [==============================] - 1141s 1ms/sample - loss: 0.1462 - accuracy: 0.9445 - val_loss: 0.1244 - val_accuracy: 0.9521
Epoch 2/5
914285/914285 [==============================] - 1124s 1ms/sample - loss: 0.1139 - accuracy: 0.9543 - val_loss: 0.1221 - val_accuracy: 0.9535
Epoch 3/5
914285/914285 [==============================] - 1143s 1ms/sample - loss: 0.1047 - accuracy: 0.9583 - val_loss: 0.1196 - val_accuracy: 0.9534
Epoch 4/5
914285/914285 [==============================] - 1118s 1ms/sample - loss: 0.0972 - accuracy: 0.9617 - val_loss: 0.1213 - val_accuracy: 0.9523
Epoch 5/5
914285/914285 [==============================] - 1094s 1ms/sample - loss: 0.0902 - accuracy: 0.9649 - val_loss: 0.1348 - val_accuracy: 0.9457


In [0]:
from tensorflow.keras.models import model_from_json

In [0]:
model_json = model.to_json()
with open('/content/Quora_text_classification_model.json','w') as file_t:
  file_t.write(model_json)

model.save_weights('/content/Quora_text_classification_model.h5')


In [0]:
with open('/content/Quora_text_classification_model.json','r') as file_r:
  model_j= file_r.read()
loaded_model=model_from_json(model_j)

In [0]:
loaded_model.load_weights('/content/Quora_text_classification_model.h5')
loaded_model.compile(optimizer='rmsprop',metrics=['accuracy'],loss='binary_crossentropy')

In [0]:
loaded_model.evaluate(S_matrix_test,y_test)

367712/391837 [===========================>..] - ETA: 1:21 - loss: 0.1348 - accuracy: 0.9457